In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [4]:
# Get the API key from the environment variable loaded by dotenv
API_KEY = os.getenv("GEMINI_API_KEY")

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Or "gemini-pro"
    temperature=0.7,           # Controls creativity (0.0 to 1.0)
    google_api_key=API_KEY     # This is how LangChain gets your API key for Gemini
)



In [5]:
class BlogState(TypedDict):
    Topic: str
    Outline: str
    Content: str

In [7]:
# Define function to generate outline

def llm_outline(state: BlogState) -> BlogState:
    Topic = state['Topic']
    prompt = f"Generate an outline for a blog post on the topic: {Topic}"
    response = model.invoke(prompt)
    state['Outline'] = response.content
    return state

In [8]:
# Define function for generating content
def llm_content(state: BlogState) -> BlogState:
    Outline = state['Outline']
    prompt = f"Generate a blog post based on the following outline:\n{Outline}"
    response = model.invoke(prompt)
    state['Content'] = response.content
    return state

In [10]:
# Define the state graph
graph = StateGraph(BlogState)

# Define Nodes
graph.add_node('llm_outline', llm_outline)
graph.add_node('llm_content', llm_content)

# Define the edges
graph.add_edge(START, 'llm_outline')
graph.add_edge('llm_outline', 'llm_content')
graph.add_edge('llm_content', END)

# Compile the workflow
workflow = graph.compile()

In [13]:
# Execute the workflow
initial_state = BlogState(Topic="The Future of AI in Everyday Life")
final_state = workflow.invoke(initial_state)
print(final_state['Topic'])
print(final_state['Outline'])
print(final_state['Content'])

The Future of AI in Everyday Life
## Blog Post Outline: The Future of AI in Everyday Life

**I. Introduction (approx. 100 words)**

* Hook: Start with a compelling anecdote or question about AI's current presence in daily life (e.g., using Siri, smart home devices).
* Briefly define AI and its current capabilities.
* Thesis statement:  AI will become increasingly integrated into our daily routines, revolutionizing various aspects of our lives, both positively and negatively.  This post will explore some key areas of this transformation.


**II. AI in the Home (approx. 200 words)**

* **Smart Homes:**  Discuss advancements in smart home technology, including more sophisticated voice assistants, proactive home maintenance (predictive repairs), personalized energy management, and integrated security systems.
* **Robotics:** Explore the potential of home robots for assistance with chores, elderly care, and companionship.  Mention limitations and ethical considerations (privacy, job displac